# Imports

In [15]:
import sys
sys.path.append('/home/rcendre/classification')
import os
import sys
import itertools
import webbrowser
from pathlib import Path
import matplotlib.pyplot as plt
from numpy import array, logspace
from scipy.stats import randint as randint
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import davies_bouldin_score
from toolbox.classification.common import Data, Folds, IO, Tools
from toolbox.classification.parameters import Dermatology, Settings
from toolbox.IO import dermatology
from toolbox.models.builtin import Applications
from toolbox.models.models import DecisionVotingClassifier, ScoreVotingClassifier
from toolbox.transforms.common import PredictorTransform
from toolbox.transforms.labels import OrderedEncoder
from toolbox.transforms.images import DistributionImageTransform, DWTImageTransform, FourierImageTransform, HaralickImageTransform, SpatialImageTransform
from toolbox.views.common import Views, ViewsTools
from toolbox.views.images import ImagesViews
from IPython.display import display
import warnings
warnings.filterwarnings('ignore') 

# Parameters

In [16]:
# Advanced parameters
data_type = 'Full'
extractor = 'Transfer'
validation = 4
settings = Settings.get_default_dermatology()

# Inputs

In [17]:
inputs = IO.load(f'Features.pickle')

In [18]:
inputs = inputs[(inputs['Binary_Diagnosis'] == 'Benign') |(inputs['Binary_Diagnosis'] == 'Malignant')].reset_index(drop=True)

# Low level predictions

In [19]:
low_file = f'Low_Supervised_Transfer.pickle'

In [20]:
low_folds = [([1, 2], [3]),
             ([2, 3], [4]),
             ([3, 4], [1]),
             ([4, 1], [2])]

In [21]:
# Add scaling step
supervised = Pipeline([ ('scale', MinMaxScaler()), ('clf', SVC(kernel='linear', class_weight='balanced', probability=True))])
supervised_params = {'clf__C': logspace(-2, 3, 6).tolist()}

In [22]:
# Transform standard inputs
Tools.fit_predict(inputs, {'datum': extractor, 'label_encode': 'LesionEncode'}, 
                  supervised, 'Supervised', folds=low_folds, distribution=supervised_params)

In [23]:
all_image = [True] * len(inputs.index)
single_image = inputs['ID_Image'] == '0M'
Data.build_bags(inputs, single_image, 'ID_Lesion', all_image, 'ID_Lesion', f'Supervised_{Tools.PREDICTION}')
Data.build_bags(inputs, single_image, 'ID_Lesion', all_image, 'ID_Lesion', f'Supervised_{Tools.PROBABILITY}')
inputs = inputs[single_image].reset_index()

In [24]:
# Save
IO.save(inputs, low_file)

# High level predictions

In [25]:
inputs = IO.load(low_file)

In [26]:
high_file = f'High_Supervised_Transfer.pickle'

In [27]:
high_folds = [([4], [1]),
             ([1], [2]),
             ([2], [3]),
             ([3], [4])]

In [28]:
# Decisions
Tools.evaluate(inputs, {'datum': f'Supervised_{Tools.PREDICTION}', 'label_encode': 'LesionEncode'}, DecisionVotingClassifier(mode='at_least_one'), 'D_ALO', folds=high_folds)
Tools.evaluate(inputs, {'datum': f'Supervised_{Tools.PREDICTION}', 'label_encode': 'LesionEncode'}, DecisionVotingClassifier(mode='dynamic_thresh'), 'D_DYN', folds=high_folds)

# Score
Tools.evaluate(inputs, {'datum': f'Supervised_{Tools.PROBABILITY}', 'label_encode': 'LesionEncode'}, ScoreVotingClassifier(low='max', high='dynamic'), 'S_MaxD', folds=high_folds)
Tools.evaluate(inputs, {'datum': f'Supervised_{Tools.PROBABILITY}', 'label_encode': 'LesionEncode'}, ScoreVotingClassifier(low='mean', high='dynamic'), 'S_MeanD', folds=high_folds)

# High Level
IO.save(inputs, high_file)

[0.62068966 0.1627907 ]
[0.625  0.4375]d...
[0.57142857 0.33333333]
[0.80851064 0.20930233]


# Analysis

## Scores and ROC

In [29]:
from IPython.display import HTML
from IPython.display import display

inputs = IO.load(high_file)

diagnosis_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])
# ROC Curve
ViewsTools.plot_size((8,8))
high_models = ['D_ALO', 'D_DYN', 'S_MaxD', 'S_MeanD']
for name in high_models: 
    #Label
    display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                        title=[f'Test - {name}'])))

,precision,recall,f1-score,support
Benign,0.92±0.07,0.28±0.11,0.43±0.15,86.00±6.18
Malignant,0.69±0.10,0.99±0.03,0.81±0.08,138.00±4.56
accuracy,0.71±0.09,0.71±0.09,0.71±0.09,0.71±0.09
macro avg,0.80±0.08,0.63±0.06,0.62±0.10,224.00±4.30
weighted avg,0.78±0.06,0.71±0.09,0.66±0.11,224.00±4.30


,precision,recall,f1-score,support
Benign,0.85±0.10,0.71±0.15,0.77±0.08,86.00±6.18
Malignant,0.84±0.07,0.92±0.07,0.88±0.04,138.00±4.56
accuracy,0.84±0.04,0.84±0.04,0.84±0.04,0.84±0.04
macro avg,0.84±0.03,0.81±0.05,0.82±0.05,224.00±4.30
weighted avg,0.84±0.03,0.84±0.04,0.84±0.05,224.00±4.30


,precision,recall,f1-score,support
Benign,0.83±0.10,0.58±0.22,0.68±0.18,86.00±6.18
Malignant,0.78±0.10,0.93±0.05,0.85±0.05,138.00±4.56
accuracy,0.79±0.07,0.79±0.07,0.79±0.07,0.79±0.07
macro avg,0.81±0.02,0.75±0.09,0.77±0.11,224.00±4.30
weighted avg,0.80±0.03,0.79±0.07,0.79±0.10,224.00±4.30


,precision,recall,f1-score,support
Benign,0.85±0.14,0.66±0.15,0.75±0.07,86.00±6.18
Malignant,0.82±0.09,0.93±0.08,0.87±0.04,138.00±4.56
accuracy,0.83±0.04,0.83±0.04,0.83±0.04,0.83±0.04
macro avg,0.83±0.04,0.80±0.05,0.81±0.05,224.00±4.30
weighted avg,0.83±0.02,0.83±0.04,0.82±0.05,224.00±4.30
